In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import warnings;
warnings.simplefilter('ignore')

In [2]:
md = pd.read_csv('C:/Users/user/Desktop/인공지능 개발자 양성과정/캐글 영화 추천 시스템/data/movies_metadata.csv')
md.info()
md = md.drop([19730, 29503, 35587])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [3]:
# genres 추출 여러 요소를 갖는 리스트 형태
print(md['genres'])

md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x:[i['name']for i in x] if isinstance(x, list) else [])

print(md['genres'])

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45463, dtype: object
0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
4546

In [4]:
# null 값 제외하고 int 자료형으로 변경 후 평균 구하기 => C

print(md['vote_count'].head())
print(md['vote_average'].head())

vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')
C = vote_averages.mean()

print(C)

# 최소한 95% 이상의 사람의 평점을 받은 영화여야 됨 => m
m = vote_counts.quantile(0.95)
print(m)

0    5415.0
1    2413.0
2      92.0
3      34.0
4     173.0
Name: vote_count, dtype: float64
0    7.7
1    6.9
2    6.5
3    6.1
4    5.7
Name: vote_average, dtype: float64
5.244896612406511
434.0


In [5]:
# 년도만 추출
print(md['release_date'])

md['year'] = pd.to_datetime(md['release_date'], errors = 'coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

print(md['year'])

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45461           NaN
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45463, dtype: object
0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45461     NaT
45462    2011
45463    2003
45464    1917
45465    2017
Name: year, Length: 45463, dtype: object


In [6]:
# 평점 count 434 이상 & null 값 아닌 데이터셋 추출
print (md.shape)
qualified = md[(md['vote_count'] >= m) & 
(md['vote_count'].notnull()) & 
(md['vote_average'].notnull())][['title', 'year', 
'vote_count', 'vote_average', 'popularity', 'genres']] 
qualified['vote_count'] = qualified['vote_count'].astype('int') 
qualified['vote_average'] = qualified['vote_average'].astype('int')

print(qualified.shape)



(45463, 25)
(2274, 6)


In [7]:
# weighted rating 구하는 함수
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['wr'] = qualified.apply(weighted_rating, axis=1) 
qualified = qualified.sort_values('wr', ascending=False).head(250)

print(qualified.head(15))


                                                   title  year  vote_count  \
15480                                          Inception  2010       14075   
12481                                    The Dark Knight  2008       12269   
22879                                       Interstellar  2014       11187   
2843                                          Fight Club  1999        9678   
4863   The Lord of the Rings: The Fellowship of the Ring  2001        8892   
292                                         Pulp Fiction  1994        8670   
314                             The Shawshank Redemption  1994        8358   
7000       The Lord of the Rings: The Return of the King  2003        8226   
351                                         Forrest Gump  1994        8147   
5814               The Lord of the Rings: The Two Towers  2002        7641   
256                                            Star Wars  1977        6778   
1225                                  Back to the Future  1985  

In [8]:
# genre를 melting 형태로 변경, md와 join
s = md.apply(lambda x: 
pd.Series(x['genres']),axis=1).stack().reset_index(level=1, 
drop=True)
s.name = 'genre'
print(s)

gen_md = md.drop('genres', axis=1).join(s) 

print(gen_md.head(10)[['title', 'vote_average', 'vote_count', 'year','genre']])


0        Animation
0           Comedy
0           Family
1        Adventure
1          Fantasy
           ...    
45461       Family
45462        Drama
45463       Action
45463        Drama
45463     Thriller
Name: genre, Length: 91094, dtype: object
               title  vote_average  vote_count  year      genre
0          Toy Story           7.7      5415.0  1995  Animation
0          Toy Story           7.7      5415.0  1995     Comedy
0          Toy Story           7.7      5415.0  1995     Family
1            Jumanji           6.9      2413.0  1995  Adventure
1            Jumanji           6.9      2413.0  1995    Fantasy
1            Jumanji           6.9      2413.0  1995     Family
2   Grumpier Old Men           6.5        92.0  1995    Romance
2   Grumpier Old Men           6.5        92.0  1995     Comedy
3  Waiting to Exhale           6.1        34.0  1995     Comedy
3  Waiting to Exhale           6.1        34.0  1995      Drama


In [12]:
# 장르, 퍼센트 옵션 추가
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']] 
    qualified['vote_count'] = qualified['vote_count'].astype('int') 
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(lambda x: (x['vote_count'] / (x['vote_count'] + m) * x['vote_average']) + (m / (m + x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified 
print(build_chart('Romance').head(15))


                             title  year  vote_count  vote_average popularity  \
10309  Dilwale Dulhania Le Jayenge  1995         661             9     34.457   
351                   Forrest Gump  1994        8147             8    48.3072   
876                        Vertigo  1958        1162             8    18.2082   
40251                   Your Name.  2016        1030             8  34.461252   
883               Some Like It Hot  1959         835             8    11.8451   
1132               Cinema Paradiso  1988         834             8     14.177   
19901                     Paperman  2012         734             8    7.19863   
37863                  Sing Street  2016         669             8  10.672862   
882                  The Apartment  1960         498             8    11.9943   
38718               The Handmaiden  2016         453             8  16.727405   
3189                   City Lights  1931         444             8    10.8915   
24886             The Way He

In [18]:
links_small = pd.read_csv('C:/Users/user/Desktop/인공지능 개발자 양성과정/캐글 영화 추천 시스템/data/links_small.csv')
print(links_small.head())



   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


In [22]:
# small data 사용
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

md['id'] = md['id'].astype('int')

smd2 = md[md['id'].isin(links_small)] 
print(smd2.shape)

(9099, 25)


In [23]:
# 영화 text 정보 column 생성 
smd2['tagline'] = smd2['tagline'].fillna('') 
smd2['description'] = smd2['overview'] + smd2['tagline']
smd2['description'] = smd2['description'].fillna('') 
print(smd2[['description']])

                                             description
0      Led by Woody, Andy's toys live happily in his ...
1      When siblings Judy and Peter discover an encha...
2      A family wedding reignites the ancient feud be...
3      Cheated on, mistreated and stepped on, the wom...
4      Just when George Banks has recovered from his ...
...                                                  ...
40224  From the mind behind Evangelion comes a hit la...
40503  The band stormed Europe in 1963, and, in 1964,...
44821  When Molly Hale's sadness of her father's disa...
44826  All your favorite Pokémon characters are back,...
45265  While holidaying in the French Alps, a Swedish...

[9099 rows x 1 columns]


In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity 

In [41]:
# tfidf 적용
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd2['description'])
print(tfidf_matrix.shape)

(9099, 268124)


In [43]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

print(cosine_sim)

smd2 = smd2.reset_index() 
titles = smd2['title']
indices = pd.Series(smd2.index, index=smd2['title']) 
print(indices)

[[1.         0.00680476 0.         ... 0.         0.00344913 0.        ]
 [0.00680476 1.         0.01531062 ... 0.00357057 0.00762326 0.        ]
 [0.         0.01531062 1.         ... 0.         0.00286535 0.00472155]
 ...
 [0.         0.00357057 0.         ... 1.         0.07811616 0.        ]
 [0.00344913 0.00762326 0.00286535 ... 0.07811616 1.         0.        ]
 [0.         0.         0.00472155 ... 0.         0.         1.        ]]
title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
Shin Godzilla                                         9094
The Beatles: Eight Days a Week - The Touring Years    9095
Pokémon: Spell of the Unknown                         9096
Pokémon 4Ever: Cele

In [56]:
# 영화 제목이 들어오면 해당 열에서 가장 유사도 높은 순으로 정렬 후 인덱스 뽑기
# contents based recommender
def get_recommendations_based_description(title, smd2):
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(smd2['description']) 
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    
    smd2 = smd2.reset_index()
    indices = pd.Series(smd2.index, index=smd2['title']) 
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] 
    movie_indices = [i[0] for i in sim_scores]
    return smd2.iloc[movie_indices][['title', 'vote_count','vote_average', 'year']]

print(get_recommendations_based_description('Toy Story', smd2).head(10))


ValueError: cannot insert level_0, already exists